In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math 
import shutil
import glob


In [3]:
# To count the numbers of images in the respective classes 0- Tumor 1- healthy
ROOT_DIR ="../Untitled Folder 6/Training"
number_of_images ={}

for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

In [4]:
number_of_images.items()

dict_items([('no_tumor', 114), ('pituitary_tumor', 171)])

In [5]:
os.listdir("../Untitled Folder 6/Training")

['no_tumor', 'pituitary_tumor']

# Splitting the Data
70% for training
15% for validating
15% for testing


In [6]:
def dataFolder(p, split):
  # we create a train folder

  if not os.path.exists("./"+p):
    os.mkdir("./"+p)

    for dir in os.listdir(ROOT_DIR):
      os.makedirs("./"+p+"/"+dir)
      for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR, dir) ) , 
                                  size = (math.floor(split*number_of_images[dir]) -5),
                                  replace=False ):
        O = os.path.join(ROOT_DIR,dir,img)
        D = os.path.join("./"+p, dir)
        shutil.copy(O,D)
        os.remove(O)
  else:
    print( f"{p}Folder exsist")

In [7]:
dataFolder("train", 0.7)

trainFolder exsist


In [8]:
dataFolder("val", 0.15)

valFolder exsist


In [9]:
dataFolder("test", 0.15)

testFolder exsist


In [10]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras

In [11]:
#CNN model

model = Sequential()

model.add(Conv2D(filters= 16, kernel_size= (3,3), activation= 'relu', input_shape = (224,224,3) ))

model.add(Conv2D(filters= 36 , kernel_size= (3,3), activation= 'relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 64 , kernel_size= (3,3), activation= 'relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 128 , kernel_size= (3,3), activation= 'relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate= 0.25))

model.add(Flatten())
model.add(Dense(units=64 ,activation= 'relu'))
model.add(Dropout(rate= 0.25 ))
model.add(Dense(units= 1, activation= 'sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 36)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       7

In [12]:
model.compile(optimizer='adam', loss= keras.losses.binary_crossentropy, metrics =['accuracy'])

# Preparing the Data


In [13]:
from keras.layers.preprocessing.image_preprocessing import HORIZONTAL
def preprocessingImage1(path):
  """
  input : Path
  output : Pre processed image
  """
  image_data = ImageDataGenerator(zoom_range= 0.2, shear_range= 0.2, rescale= 1/225, horizontal_flip=True)
  image = image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size = 32, class_mode = 'binary')

  return image

In [14]:
path = "../Untitled Folder 6/train"
train_data = preprocessingImage1(path)

Found 844 images belonging to 2 classes.


In [15]:
def preprocessingImage2(path):
  """
  input : Path
  output : Pre processed image
  """
  image_data = ImageDataGenerator(rescale= 1/225,)
  image = image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size = 32, class_mode = 'binary')

  return image 

In [16]:
path = '../Untitled Folder 6/test'
test_data = preprocessingImage2(path)

Found 173 images belonging to 2 classes.


In [17]:
path = '../Untitled Folder 6/val'
val_data = preprocessingImage2(path)

Found 173 images belonging to 2 classes.


In [18]:
# check point

from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor="val_accuracy", min_delta= 0.01 , patience= 5, verbose= 1, mode = 'auto')

mc = ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5", verbose= 1, save_best_only= True, mode ='auto')
cd = [es,mc]

In [ ]:
hs = model.fit_generator(generator= train_data,
                         steps_per_epoch= 8 ,
                         epochs=30 ,
                         verbose= 1,
                         validation_data= val_data,
                         validation_steps= 4, 
                         callbacks = cd)

Epoch 1/30
8/8 [==============================] - ETA: 0s - loss: 0.8123 - accuracy: 0.6144
Epoch 1: val_accuracy improved from -inf to 0.67969, saving model to .\bestmodel.h5
8/8 [==============================] - 33s 4s/step - loss: 0.8123 - accuracy: 0.6144 - val_loss: 0.6284 - val_accuracy: 0.6797
Epoch 2/30
3/8 [==========>...................] - ETA: 18s - loss: 0.5860 - accuracy: 0.7604

In [ ]:
h = hs.history
h.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = "black")

plt.title("acc vs val-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "black")

plt.title("loss vs val-loss")
plt.show()

In [ ]:
##Accuracy
from keras.models import load_model

model = load_model("../Untitled Folder 6/bestmodel.h5")

In [ ]:
acc = model.evaluate_generator(test_data)[1]

print(f"The accuracy of the model is {acc*100} %")

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
path = "../Untitled Folder 6/yes/Y52.jpg"

img = load_img(path, target_size = (224,224))
input_arr = img_to_array(img)/255

plt.imshow(input_arr)
plt.show()

input_arr.shape

input_arr = np.expand_dims(input_arr, axis= 0)

pred = (model.predict(input_arr) > 0.5).astype("int32")
pred

if pred ==0:
  print("The MRI shows no Tumor")
else:
  print("The MRI shows  Tumor")

In [ ]:
train_data.class_indices